In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt
import openpyxl
import pandas as pd
from matplotlib import image
import cv2
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
import sys

from numpy.random import seed
seed(2)

import tensorflow
tensorflow.random.set_seed(2)

class My_Custom_Model:
    
    
    def __init__ (self, I_want_to_train_models, path_csv, path_images, 
                  img_width, img_height, dataAug,
                  loss_function,
                  type_image,  model_conv, model_type,                  
                  freeze, drop, neurons_last_layer, initializer_seed, metrics, 
                  model_directory, patience, 
                  print_report, 
                  p_to_save):

        self.path_csv = path_csv
        self.type_image = type_image
        self.path_images = path_images
        self.img_width= img_width
        self.img_height= img_height
        self.dataAug= dataAug
        self.model_conv= model_conv
        self.model_type=model_type
        self.freeze= freeze
        self.neurons_last_layer= neurons_last_layer
        self.initializer_seed= initializer_seed
        self.lossf= loss_function
        self.metrics= metrics
        self.p_to_save= p_to_save
        self.model_directory= model_directory
        self.patience= patience
        self.print_report= print_report
        self.I_want_to_train_models=I_want_to_train_models
        
    
    def load_data(self):
        if not os.path.exists(self.model_directory + 'Input/'): os.mkdir(self.model_directory + 'Input/')  

        if (self.model_type == 'Hybrid' and self.type_image=='2D') or self.type_image=='3D':
            
            self.partition='Train'
            self.X_train_image, self.y_train = self.load_images()
            if self.model_type == 'Hybrid': self.X_train_dataf = self.load_dataframe() 

            self.partition='Validation'
            self.X_val_image, self.y_val = self.load_images()
            if self.model_type == 'Hybrid': self.X_val_dataf = self.load_dataframe()

            self.partition='Test'
            self.X_test_image, self.y_test  = self.load_images()
            if self.model_type == 'Hybrid':  self.X_test_dataf = self.load_dataframe()
            if self.model_type == 'Hybrid': print('[INFO] DEMOGRAFIC DATA: TRAIN SHAPE: '+ str(self.X_train_dataf.shape)+ ' - VAL SHAPE: '+ str(self.X_val_dataf.shape)+ ' - TEST SHAPE: ' + str(self.X_test_dataf.shape))
            
            print('[INFO] IMAGES: TRAIN SHAPE: '+ str(self.X_train_image.shape)+ ' - VAL SHAPE: '+ str(self.X_val_image.shape)+ ' - TEST SHAPE: ' + str(self.X_test_image.shape))
            print('[INFO] LABELS: TRAIN SHAPE: '+ str(self.y_train.shape)+ ' - VAL SHAPE: '+ str(self.y_val.shape)+ ' - TEST SHAPE: ' + str(self.y_test.shape))
        
        if (self.model_type == 'Images' and self.type_image=='2D'):
            self.partition='Train'
            self.train_generator = self.load_images_ImageDataGenerator()
            
            self.partition='Validation'
            self.val_generator = self.load_images_ImageDataGenerator()

            self.partition='Test'
            self.test_generator  = self.load_images_ImageDataGenerator()
        
            
        
    def load_images_ImageDataGenerator(self):
        
        if self.partition == 'Train':
            self.csv = pd.read_csv(self.path_csv + 'TRAIN1_NUEVOS.csv', sep=',')

        elif self.partition == 'Validation':
            self.csv = pd.read_csv(self.path_csv +'VAL1_NUEVOS.csv', sep=',')
            
        else:
            self.csv = pd.read_csv(self.path_csv + 'TEST15_NUEVOS.csv', sep=',') 
            self.csv_test = self.csv 
            
        self.csv['Label']=self.csv['Label'].astype(str)
            
                
        self.class_mode='binary' if self.neurons_last_layer < 2 else 'categorical'
        
        if self.dataAug == True and self.partition == 'Train':
            datagen = ImageDataGenerator(rescale=1./255, rotation_range=90, horizontal_flip=False, brightness_range=[0.3,0.9])
        else:
            datagen = ImageDataGenerator(rescale=1./255)
        
        self.shuffle = True if self.partition == 'Train' else False
        generator = datagen.flow_from_dataframe(self.csv, directory=self.path_images, x_col='Patient',
                                                    y_col='Label', batch_size=self.batch_size, class_mode= self.class_mode,
                                                    color_mode='rgb', target_size=(self.img_width, self.img_height),
                                                    shuffle=self.shuffle)
        return generator
        
    def load_images(self):
        
        if self.type_image=='3D':
            if self.partition == 'Train':
                self.csv = pd.read_csv(self.path_csv + 'TRAIN_PATIENT_random_normalizado.csv', sep=',')

            elif self.partition == 'Validation':
                self.csv = pd.read_csv(self.path_csv +'VAL_PATIENT_random_normalizado.csv', sep=',')
            else:
                self.csv = pd.read_csv(self.path_csv + 'TEST15_PATIENT_NUEVO_random_normalizado.csv', sep=',') 
                self.csv_test = self.csv 
                
        elif self.type_image=='2D':
            if self.partition == 'Train':
                self.csv = pd.read_csv(self.path_csv + 'TRAIN1_NUEVOS.csv', sep=',')

            elif self.partition == 'Validation':
                self.csv = pd.read_csv(self.path_csv +'VAL1_NUEVOS.csv', sep=',')
            
            else:
                self.csv = pd.read_csv(self.path_csv + 'TEST15_NUEVOS.csv', sep=',') 
                self.csv_test =  pd.read_csv(self.path_csv + 'TEST15_NUEVOS.csv', sep=',')
            
        X = []
        for filename in self.csv['Patient']:
            if self.type_image == '3D':
                img_data = np.load(self.path_images + filename +'.npy')
                img_data = self.resize(img_data)
                    
            elif self.type_image == '2D':
                # load image
                img_data = image.imread(self.path_images + filename)
                img_data = self.resize(img_data)
                    
            X.append(img_data)

        X = np.expand_dims(np.asarray(X, dtype=np.float64), axis=4) if self.type_image == '3D' else np.stack((X,X,X), axis=3).astype('float64')
            
        return X, self.csv['Label']
    
    
    def load_dataframe(self):
        for i in range(len(self.csv['Patient'])): 
            self.csv.loc[i, 'PatientAge'] = round((self.csv['PatientAge'][i]-min(self.csv['PatientAge'])) / (max(self.csv['PatientAge']) - min(self.csv['PatientAge'])),2)
        self.csv_data = self.csv
        self.csv_data=self.csv_data.drop(['Patient','Label'], axis=1)
        self.csv_data.to_csv(self.model_directory + 'Input/'+self.partition+'_data.csv', index = False)
        return np.asarray(self.csv_data).astype('float64')


    def resize(self, img_data):    
        if self.type_image=='3D':
            n,m,l=img_data.shape
            img_p=list(img_data)

            matriz_float = np.zeros((m,l))
            matriz_float = np.float32(matriz_float)
            self.depth=45
            for ind in range(n,self.depth):
                img_p.append(matriz_float)
            img_p=np.array(img_p)


            n,m,l=img_p.shape
            img_data=np.zeros([self.img_width,self.img_height, n])

            for i in range(0,n):
                img_data[:,:,i]=cv2.resize(img_p[i,:,:],(self.img_width,self.img_height,))
                
        elif self.type_image == '2D':
            img_data = cv2.resize(img_data, (self.img_width, self.img_height))
            img_data = img_data/255.
            if len(img_data.shape) == 3: img_data=img_data[:,:,0]

        return img_data
    
    
    def get_model_CSIC2D(self):
        inputs = keras.Input((self.img_width, self.img_height, 3))    
        x = keras.layers.Conv2D(filters=16, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed),   activation="relu")(inputs) 
        x = keras.layers.Conv2D(filters=16, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed),  activation="relu")(x)
        x = keras.layers.MaxPool2D(pool_size=2)(x)
        x = keras.layers.Dropout(self.drop)(x)
        
        x = keras.layers.Conv2D(filters=32, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed),  activation="relu")(x)
        x = keras.layers.Conv2D(filters=32, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed),  activation="relu")(x)
        x = keras.layers.MaxPool2D(pool_size=2)(x)

        outputs = keras.layers.Conv2D(filters=64, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed), activation="relu", name="last_layer")(x)
        self.base_model = keras.Model(inputs, outputs, name="CSIC 2D MODEL")
            
    
    def get_model_CSIC3D(self):    
        inputs = keras.Input((self.img_width, self.img_height, self.depth, 1))
        x = keras.layers.Conv3D(filters=16, kernel_size=3,  kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed), activation="relu",use_bias=False)(inputs)
        x = keras.layers.Conv3D(filters=16, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed), activation="relu",use_bias=False)(x)
        x = keras.layers.MaxPool3D(pool_size=2)(x)
        x = keras.layers.SpatialDropout3D(self.drop)(x)

        x = keras.layers.Conv3D(filters=32, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed), activation="relu",use_bias=False)(x)
        x = keras.layers.Conv3D(filters=32, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed), activation="relu",use_bias=False)(x)
        x = keras.layers.MaxPool3D(pool_size=2)(x)

        outputs = keras.layers.Conv3D(filters=64, kernel_size=3, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed),  activation="relu", name="last_layer",use_bias=False)(x)
        self.base_model = keras.Model(inputs, outputs, name="CSIC 3D MODEL")

    def base_model(self):        
        if self.type_image == '2D':
            if self.model_conv == 'VGG16':
                self.base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape= (self.img_width, self.img_height, 3)) 

                
                if self.freeze == False:
                    for layer in self.base_model.layers:
                        layer.trainable = True
                    print('No frozen layers')

                elif self.freeze == True:
                    for layer in self.base_model.layers:
                        layer.trainable = False
                    print ('All layers frozen')
                else:        
                    self.freeze = int(input("How many layers do you want to freeze?  - (options: 1 - "+str(len(self.base_model.layers))+"): "))
                    for layer in self.base_model.layers[0:self.freeze]:
                            layer.trainable = False
                            print('Layer ' + layer.name + ' frozen...')
                    for layer in self.base_model.layers[self.freeze:len(self.base_model.layers)]:
                            layer.trainable = True
                            print('Layer ' + layer.name + ' unfrozen...')  

            elif self.model_conv == 'CSIC':
                self.get_model_CSIC2D()
                
        elif self.type_image == '3D':
            if self.model_conv == 'CSIC':
                self.get_model_CSIC3D()
            else:
                print('Model unavailable')
                
                    
    def get_model_hybrid(self):
        n, m = self.X_train_dataf.shape
        input_data_hybrid = keras.Input(shape=(m,))
        y=input_data_hybrid
        #y = keras.layers.Dense(10, kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed),activation = 'relu', use_bias=False)(input_data_hybrid) if self.type_image == '3D' else input_data_hybrid
        self.model_hybrid = keras.Model(inputs=input_data_hybrid, outputs=y)
        self.model_top = keras.layers.concatenate([self.model_top, self.model_hybrid.output])
        
        
    def model_top(self):    
        self.model_top = self.base_model.output
        
        # Definir top model
        if self.top_m == 'GMP':
            self.model_top = keras.layers.GlobalMaxPooling3D()(self.model_top) if self.type_image == '3D' else keras.layers.GlobalMaxPooling2D()(self.model_top)

        elif self.top_m == 'GAP':
            self.model_top = keras.layers.GlobalAveragePooling3D()(self.model_top) if self.type_image == '3D' else keras.layers.GlobalAveragePooling2D()(self.model_top)

        elif self.top_m == "FD":
            self.model_top = keras.layers.Flatten()(self.model_top)
            self.model_top = keras.layers.Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.HeUniform(seed=self.initializer_seed))(self.model_top)
             
                
        if self.model_type == 'Hybrid': self.get_model_hybrid()
    
        
        if self.neurons_last_layer == 1:
            self.model_prediction = keras.layers.Dense(self.neurons_last_layer, activation='sigmoid',
                                                kernel_initializer= tf.keras.initializers.GlorotUniform(seed=self.initializer_seed))(self.model_top)
                                                                           
            if self.lossf == 'categorical_crossentropy':
                self.lossf == 'binary_crossentropy'
                
        if self.neurons_last_layer > 1:
            self.model_prediction = keras.layers.Dense(self.neurons_last_layer, activation='softmax',
                                                  kernel_initializer= tf.keras.initializers.GlorotUniform(seed=self.initializer_seed))(self.model_top)
            if self.lossf == 'binary_crossentropy':
                self.lossf == 'categorical_crossentropy'
        
        #if self.model_type == 'Hybrid': self.model = keras.Model(inputs=[self.base_model.input, self.model_hybrid.input], outputs=self.model_prediction)
        self.model = keras.Model(inputs=[self.base_model.input, self.model_hybrid.input], outputs=self.model_prediction) if self.model_type == 'Hybrid' else keras.Model(inputs=self.base_model.input, outputs=self.model_prediction)
        self.model.summary()
        
    def model_compilation (self):
        if self.topt == 'SGD':
            self.opt= keras.optimizers.SGD (learning_rate= self.lr)
        elif self.topt == 'Adam':
            self.opt= keras.optimizers.Adam (learning_rate= self.lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        elif self.topt == 'Adadelta':
            self.opt= keras.optimizers.Adadelta (learning_rate= self.lr)
        elif self.topt == 'Adagrad':
            self.opt= keras.optimizers.Adagrad (learning_rate= self.lr)

        self.model.compile (optimizer=self.opt, loss=self.lossf, metrics= self.metrics)
           
            
            
    def model_fit(self):
        self.my_callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=self.patience)]

        if (self.model_type == 'Hybrid' and self.type_image == '2D') or self.type_image == '3D':
            if self.model_type == 'Hybrid':
                train_data = [self.X_train_image, self.X_train_dataf]  
                validation_data =  [self.X_val_image, self.X_val_dataf] 
            elif self.type_image == '3D':
                train_data = self.X_train_image 
                validation_data =  self.X_val_image
            self.history = self.model.fit(train_data, self.y_train,
                                          batch_size = self.batch_size, epochs = self.epochs,
                                          validation_data = (validation_data, self.y_val),
                                          callbacks = self.my_callbacks, verbose=1)
        elif (self.model_type == 'Images' and self.type_image == '2D'):
            self.history = self.model.fit_generator(generator=self.train_generator, steps_per_epoch=self.train_generator.n//self.train_generator.batch_size,
                                epochs=self.epochs, verbose=1, validation_data=self.val_generator,
                                validation_steps=self.val_generator.n//self.val_generator.batch_size, callbacks=self.my_callbacks)
                
                
                
    def plot_learning_curves(self):
        # Save curves training
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(self.history.history['accuracy'])
        plt.plot(self.history.history['val_accuracy'])
        plt.title('Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epochs')
        plt.ylim(0, 1)
        plt.legend(['train', 'val'], loc='best')
        plt.savefig(self.path_to_save_metrics + 'LEARNINGCURVE_Accuracy_' + self.model_name + '.png')
        plt.close()
        
        plt.figure()
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epochs')
        plt.ylim(0, 1)
        plt.legend(['train', 'val'], loc='best')
        plt.savefig(self.path_to_save_metrics + 'LEARNINGCURVE_Loss_' + self.model_name + '.png')
        plt.close()
        
    def save_images_fpfntptn(self, path_list_tps_fps_fns_tns):
        
        for idir in ['FNS','FPS','TNS','TPS']:
            if not os.path.exists(path_list_tps_fps_fns_tns+'/'+idir+'/'): os.mkdir(path_list_tps_fps_fns_tns+'/'+idir+'/') 
            csv = pd.read_csv(path_list_tps_fps_fns_tns+'/' + idir +'.csv', sep=',')
            if idir=='FNS': 
                y_true = 1  
                y_pred = 0

            if idir=='TPS': 
                y_true = 1  
                y_pred = 1
            if idir=='FPS' :
                y_true = 0 
                y_pred = 1

            if idir=='TNS':
                y_true = 0 
                y_pred = 0
                
            for filename in csv['Marca temporal']:
                img_data = Image.open(self.path_images+filename).convert("L")
                plt.imshow(img_data, cmap='gray')
                plt.axis('off')

                plt.title(filename+' - Sex:'+str(csv['PatientSex'][i])+ ' - Age'+str(csv['PatientAge'][i])+' - Label: '+str(y_true)+' Prediction: '+str(y_pred))
                plt.savefig(path_list_tps_fps_fns_tns+'/'+idir+'/'+ filename +'_Sex'+str(csv['PatientSex'][i])+'_Age'+str(csv['PatientAge'][i])+'_Label'+str(y_true)+'_Prediction'+str(y_pred)+'.jpg')
                            
    def list_fp_fn_tp_tn(self):
        
        if not os.path.exists(self.path_to_save_metrics+'/LIST_TPS_FPS_FNS_TNS/'): os.mkdir(self.path_to_save_metrics+'/LIST_TPS_FPS_FNS_TNS/')  
        path_list_tps_fps_fns_tns=self.path_to_save_metrics+'/LIST_TPS_FPS_FNS_TNS/'+self.model_parameters+'_'+str(self.threshold_plot_confusion_matrix_sex)
        if not os.path.exists(path_list_tps_fps_fns_tns): os.mkdir(path_list_tps_fps_fns_tns)  
        
        
        y_pred_threshold= self.predictions
        
        for ithreshold in [self.threshold_plot_confusion_matrix_sex]:
            y_pred_threshold[self.predictions >= ithreshold] = 1                
            y_pred_threshold[self.predictions < ithreshold] = 0

            
            itps, ifps, ifns, itns = 0, 0, 0, 0
            

            TPS, FPS, FNS, TNS = self.csv_test.iloc[0:0], self.csv_test.iloc[0:0], self.csv_test.iloc[0:0], self.csv_test.iloc[0:0]
            
            for i in range(len(y_pred_threshold)):
                pred = 0 if int(y_pred_threshold[i]) == 0 else 1
                    
                if int(pred) == 1 and int(self.y_true[i]) == 1:
                    TPS.loc[itps, :]=self.csv_test.loc[i, :]
                    itps=itps+1

                if int(pred) == 1 and int(self.y_true[i]) == 0:
                    FPS.loc[ifps, :]=self.csv_test.loc[i, :]
                    ifps = ifps+1                  
                    
                if int(pred)== 0 and int(self.y_true[i]) == 1:
                    FNS.loc[ifns, :]=self.csv_test.loc[i, :]
                    ifns = ifns+1     

                if int(pred) == 0 and int(self.y_true[i]) == 0:
                    TNS.loc[itns, :]=self.csv_test.loc[i, :]
                    itns = itns+1
                    
        TPS.to_csv(path_list_tps_fps_fns_tns+'/TPS.csv', index = False)
        FPS.to_csv(path_list_tps_fps_fns_tns+'/FPS.csv', index = False)
        FNS.to_csv(path_list_tps_fps_fns_tns+'/FNS.csv', index = False)
        TNS.to_csv(path_list_tps_fps_fns_tns+'/TNS.csv', index = False)
        
        save_images=  str(input("Do you want to save TPS, FPS, FNS y TNS images?  - (options: Yes, No): "))
        if save_images == 'Yes': self.save_images_fpfntptn(path_list_tps_fps_fns_tns)
        
    def folders(self):
        if not os.path.exists(self.model_directory + '/'): os.mkdir(self.model_directory + '/')  
        if not os.path.exists(self.model_directory + '/'+self.type_image +'/'): os.mkdir(self.model_directory + '/'+self.type_image +'/')  
        if not os.path.exists(self.model_directory + '/'+self.type_image +'/'+self.model_conv +'/'): os.mkdir(self.model_directory + '/'+self.type_image +'/'+self.model_conv +'/')  
        if not os.path.exists(self.model_directory + '/'+self.type_image +'/'+self.model_conv +'/'+self.model_type+'/'): os.mkdir(self.model_directory + '/'+self.type_image +'/'+self.model_conv +'/'+self.model_type+'/')
        
        self.path_to_save_metrics = self.model_directory + '/'+self.type_image +'/'+self.model_conv +'/'+self.model_type+'/'
        
        if self.I_want_to_train_models == 'Yes': self.model_parameters = 'MODELDETECTION_FREEZE'+self.str(self.freeze)+'_TOPMODEL' + self.top_m + '_CLASSES' + str(self.neurons_last_layer) + '_INITIALIZER_SEED' + str(self.initializer_seed) + '_OPTIMIZER' + self.topt + '_lr' + str(self.lr) + '_LOSSF' + self.lossf + '_BATCHSIZE' + str(self.batch_size) + '_EPOCHS' + str(self.epochs)
    
    def load_model(self):
        self.model_parameters = str(input("Name of the model you want to load (without .hdf5): "))
        print(f"Name: {self.model_parameters}")
        self.model = tf.keras.models.load_model(self.path_to_save_metrics + self.model_parameters+'.hdf5')   
        
        
    def model_predict(self):
        self.folders()
        if self.I_want_to_train_models == 'No': self.load_model() 
        
        for ithreshold in list(range(1,10,1)):
            self.threshold_plot_confusion_matrix_sex = float(input("What threshold do you want to get list of FP, FN, TP and TN?  - (options: 0.1-0.9): "))
            self.threshold = ithreshold/10
            self.Error_metrics = False
            
                
            if (self.model_type == 'Hybrid' and self.type_image == '2D') or self.type_image == '3D':
                if self.model_type == 'Hybrid':
                    test_data = [self.X_test_image, self.X_test_dataf] 
                elif self.type_image == '3D':
                    test_data = self.X_test_image 
                self.predictions = self.model.predict(test_data, verbose=1)
                self.y_true = np.array(self.y_test)

                
            elif (self.model_type == 'Images' and self.type_image == '2D'):
                self.predictions = self.model.predict_generator(self.test_generator, steps= self.test_generator.n // self.test_generator.batch_size+1, verbose=1)
                self.y_true = np.array(self.test_generator.labels)
            
            self.get_auc()
            
            if self.AUC > self.p_to_save:
                self.get_metrics()

                if self.threshold==self.threshold_plot_confusion_matrix_sex: self.list_fp_fn_tp_tn()
                
                
     
    def get_auc(self):
        self.y_pred=self.predictions[:,0]
        self.AUC = roc_auc_score(y_true=self.y_true, y_score=self.y_pred)
        self.model_name = 'AUC'+str(round(self.AUC, 4)) + self.model_parameters
        
        if self.AUC > self.p_to_save: 
            self.roc_curve()
            self.bootstrapping(metrics = 'AUC')
        
        
    def get_metrics(self):      
        
        self.y_pred_threshold = self.predictions[:,0]
            
        self.y_pred_threshold[self.predictions[:,0] >= self.threshold] = 1                
        self.y_pred_threshold[self.predictions[:,0] < self.threshold] = 0

        self.bootstrapping(metrics = 'METRICS')
        
            
        if self.Error_metrics == False:                
            if self.I_want_to_train_models=='Yes':
                keras.models.save_model(self.model, self.path_to_save_metrics + self.model_name + '.hdf5')
                pd.DataFrame.from_dict(self.history.history).to_csv(self.path_to_save_metrics + self.model_name +".csv",index=False)
                self.plot_learning_curves()
        else:
            print("Error, ZeroDivisionError with all thresholds.")

                
    def roc_curve(self):
        fpr, tpr, _ = roc_curve(self.y_true, self.predictions[:,0])
        plt.plot(fpr, tpr, linewidth=2)
        plt.plot([0, 1], [0, 1], color='blue', linestyle='--', label='Baseline')
        plt.title('ROC Curve', size=12)
        plt.xlabel('False Positive Rate', size=10)
        plt.ylabel('True Positive Rate', size=10)
        plt.savefig(self.path_to_save_metrics + 'CURVEROC_' + self.model_name + '.png')
        plt.close()
        
        
    def plot_confusion_matrix(self, path_name_boost, normalize=False, title=None, cmap=plt.cm.Blues, classes= {'0': 'Health', '1': 'ICH'}):
        if not title:
            if normalize:
                title = 'Normalized confusion matrix'
            else:
                title = 'Confusion matrix, without normalization'

        # Compute confusion matrix
        cm = confusion_matrix(self.y_true, self.y_pred_threshold)
        
        if normalize: cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            
        fig, ax = plt.subplots()
        im = ax.imshow(cm, interpolation='nearest', cmap=cmap)

        ax.figure.colorbar(im, ax=ax)
        
        # We want to show all ticks...
        ax.set(xticks=np.arange(cm.shape[1]),
               yticks=np.arange(cm.shape[0]),
               # ... and label them with the respective list entries
               xticklabels=classes, yticklabels=classes,
               title=title,
               ylabel='True label',
               xlabel='Predicted label')

        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")
        plt.grid(False)

        # Loop over data dimensions and create text annotations.
        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i, j], fmt),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
        fig.tight_layout()

        ax.figure.savefig(path_name_boost+'.png')
        plt.close()
        
    
    def bootstrapping(self, metrics = 'AUC', confidence = 0.95, n_bootstraps = 1000, threshold = 0.9):
        
        rng_seed= np.random.RandomState(42) #definimos semilla de aleatorización.         
        
        if metrics == 'METRICS': 
            bootstrapped_sensitivity= []
            bootstrapped_specificity= []
            bootstrapped_precission= []
            bootstrapped_NPV= []  
            bootstrapped_accuracy= []
            bootstrapped_f1_score= []
            bootstrapped_f2_score= [] 
                
        elif metrics == 'AUC':
            bootstrapped_roc_auc = []
            
                
        for i in range(n_bootstraps): #en cada boostrap subsamplea el test y saca una AUC. 
            index= rng_seed.randint(0, self.y_true.shape[0], self.y_true.shape[0])   #0: cual es el número más pequeño que puede salir, es cero porque nuestros índices empiezan el cero. El número máximo es el índice más alto del array, osea, 256. El tercer número es el número de número que quieres que te salgan por cada vez, también 256.  
            if len(np.unique(self.y_true[index])) < 2:  #si menor de 2, no vale. np.unique devuelve un array con los valores diferentes que tú tienes. Ej. hombre o mujer tiene 1 y 2 de np unique o F y M. 
                continue
                
            if metrics == 'METRICS': 
                tps, fps, fns, tns = [], [], [], []

                for i in index:
                    pred = 0 if int(self.y_pred_threshold[i]) == 0 else 1
                    
                    if int(pred) == 1 and int(self.y_true[i]) == 1:
                        tps.append(self.y_pred_threshold[i])
                    if int(pred) == 1 and int(self.y_true[i]) == 0:
                        fps.append(self.y_pred_threshold[i]) 
                    if int(pred)== 0 and int(self.y_true[i]) == 1:
                        fns.append(self.y_pred_threshold[i]) 
                    if int(pred) == 0 and int(self.y_true[i]) == 0:
                        tns.append(self.y_pred_threshold[i]) 

                tp= len(tps)
                fp= len(fps)
                fn= len(fns)
                tn= len(tns)

                
                try:
                    sensitivity= tp/(tp+fn)
                    specificity= tn/(tn+fp)
                    precission= tp/(tp+fp)
                    NPV=tn/(tn+fn)
                    accuracy= (tp+tn)/(tp+fn+fp+tn)
                    f1_score= 2*sensitivity*precission/(sensitivity+precission)
                    f2_score= 5*sensitivity*precission/(sensitivity+4*precission)
                except ZeroDivisionError:
                    self.Error_metrics = True
                    return 

            
                bootstrapped_sensitivity.append(sensitivity)
                bootstrapped_specificity.append(specificity)
                bootstrapped_precission.append(precission)
                bootstrapped_NPV.append(NPV)
                bootstrapped_accuracy.append(accuracy)
                bootstrapped_f1_score.append(f1_score)
                bootstrapped_f2_score.append(f2_score)
                
            elif metrics == 'AUC':
                roc_auc= roc_auc_score(self.y_true[index], self.y_pred[index])  #images y predicciones. 
                bootstrapped_roc_auc.append(roc_auc)
                

        if metrics == 'METRICS': 
            bootstrapped_sensitivity= np.array(bootstrapped_sensitivity)
            bootstrapped_sensitivity.sort()
            bootstrapped_specificity= np.array(bootstrapped_specificity)
            bootstrapped_specificity.sort()
            bootstrapped_precission= np.array(bootstrapped_precission)
            bootstrapped_precission.sort()
            bootstrapped_NPV= np.array(bootstrapped_NPV)
            bootstrapped_NPV.sort()
            bootstrapped_accuracy= np.array(bootstrapped_accuracy)
            bootstrapped_accuracy.sort()
            bootstrapped_f1_score= np.array(bootstrapped_f1_score)
            bootstrapped_f1_score.sort()
            bootstrapped_f2_score= np.array(bootstrapped_f2_score)
            bootstrapped_f2_score.sort()


            b_sensitivity= np.average(bootstrapped_sensitivity)
            lower_sensitivity= bootstrapped_sensitivity[int(((1-confidence)/2) * len(bootstrapped_sensitivity))]
            upper_sensitivity= bootstrapped_sensitivity[int((confidence + ((1-confidence)/2)) * len(bootstrapped_sensitivity))]
            b_specificity= np.average(bootstrapped_specificity)
            lower_specificity= bootstrapped_specificity[int(((1-confidence)/2) * len(bootstrapped_specificity))]
            upper_specificity= bootstrapped_specificity[int((confidence + ((1-confidence)/2)) * len(bootstrapped_specificity))]
            b_precission= np.average(bootstrapped_precission)
            lower_precission= bootstrapped_precission[int(((1-confidence)/2) * len(bootstrapped_precission))]
            upper_precission= bootstrapped_precission[int((confidence + ((1-confidence)/2)) * len(bootstrapped_precission))]
            b_NPV= np.average(bootstrapped_NPV)
            lower_NPV= bootstrapped_NPV[int(((1-confidence)/2) * len(bootstrapped_NPV))]
            upper_NPV= bootstrapped_NPV[int((confidence + ((1-confidence)/2)) * len(bootstrapped_NPV))]
            b_accuracy= np.average(bootstrapped_accuracy)
            self.accuracy = round(b_accuracy, 4)
            lower_accuracy= bootstrapped_accuracy[int(((1-confidence)/2) * len(bootstrapped_accuracy))]
            upper_accuracy= bootstrapped_accuracy[int((confidence + ((1-confidence)/2)) * len(bootstrapped_accuracy))]
            b_f1_score= np.average(bootstrapped_f1_score)
            lower_f1_score= bootstrapped_f1_score[int(((1-confidence)/2) * len(bootstrapped_f1_score))]
            upper_f1_score= bootstrapped_f1_score[int((confidence + ((1-confidence)/2)) * len(bootstrapped_f1_score))]
            b_f2_score= np.average(bootstrapped_f2_score)
            lower_f2_score= bootstrapped_f2_score[int(((1-confidence)/2) * len(bootstrapped_f2_score))]
            upper_f2_score= bootstrapped_f2_score[int((confidence + ((1-confidence)/2)) * len(bootstrapped_f2_score))]  


            print('Sensitivity=%.3f, CI: lower=%.3f, upper=%.3f'%(b_sensitivity, lower_sensitivity, upper_sensitivity))
            print('Specificity=%.3f, CI: lower=%.3f, upper=%.3f'%(b_specificity, lower_specificity, upper_specificity))
            print('Precission=%.3f, CI: lower=%.3f, upper=%.3f'%(b_precission, lower_precission, upper_precission))
            print('NPV=%.3f, CI: lower=%.3f, upper=%.3f'%(b_NPV, lower_NPV, upper_NPV))
            print('Acurracy=%.3f, CI: lower=%.3f, upper=%.3f'%(self.accuracy, lower_accuracy, upper_accuracy))
            print('F1_score=%.3f, CI: lower=%.3f, upper=%.3f'%(b_f1_score, lower_f1_score, upper_f1_score))
            print('F2_score=%.3f, CI: lower=%.3f, upper=%.3f'%(b_f2_score, lower_f2_score, upper_f2_score))

            productos = [
            ('Sensitivity', b_sensitivity, lower_sensitivity, upper_sensitivity),
            ('Specificity', b_specificity, lower_specificity, upper_specificity),
            ('Precission', b_precission, lower_precission, upper_precission),
            ('NPV', b_NPV, lower_NPV, upper_NPV),
            ('Acurracy', self.accuracy, lower_accuracy, upper_accuracy),
            ('F1_score', b_f1_score, lower_f1_score, upper_f1_score),
            ('F2_score', b_f2_score, lower_f2_score, upper_f2_score)]
            
        elif metrics == 'AUC':
            sorted_roc_auc_scores= np.array(bootstrapped_roc_auc) #convertimos en array la lista de las diferentes AUCs, para utilizar la función sort y ordenarlas.
            sorted_roc_auc_scores.sort()  # ordenamos de tal forma que tenemos una curva de gauss, la media estará en medio. 


            roc_auc_bootstraping= np.average(np.array(bootstrapped_roc_auc))
            lower_roc_auc_difference= sorted_roc_auc_scores[int(((1-confidence)/2) * len(sorted_roc_auc_scores))]
            upper_roc_auc_difference= sorted_roc_auc_scores[int((confidence + ((1-confidence)/2)) * len(sorted_roc_auc_scores))]    

            print('ROC_AUC=%.3f, CI: lower=%.3f, upper=%.3f'%(roc_auc_bootstraping, lower_roc_auc_difference, upper_roc_auc_difference))

            productos = [('ROC_AUC', roc_auc_bootstraping, lower_roc_auc_difference, upper_roc_auc_difference)]
            

        wb = openpyxl.Workbook()
        hoja = wb.active
            
        # Crea la fila del encabezado con los títulos
        hoja.append(('Metrics','Value', 'CI - lower', 'CI - upper'))

        for producto in productos:
            hoja.append(producto)
            path_name_boost= self.path_to_save_metrics + '/BOOST_'+metrics+'_'+self.model_name
            if metrics == 'METRICS': path_name_boost= path_name_boost+'_threshold'+str(self.threshold)+'_ACC'+str(self.accuracy)
            
            wb.save(path_name_boost+'.xlsx')
            read_file = pd.read_excel(path_name_boost+'.xlsx')
            read_file.to_csv(path_name_boost+'.csv', index = None, header=True)
            
            if metrics == 'METRICS': self.plot_confusion_matrix(path_name_boost)      
        

    def print_data (self):
        # Si se quiere obtener información del modelo, pasar True, con cualquier otro valor no se imprime
        if self.print_report == True:
            print('Model_type: ', self.model_type, '  -  Model_conv: ', self.model_conv )
            print('\n Model_top: ', self.top_m, '\n- nclasses: ', str(self.neurons_last_layer))
            print('\n Compilation: ', self.topt, '\n- lr: ' + str(self.lr), '\n- lossf ' + self.lossf)
            print(self.model.summary())
            print('\n Model_fit: \n- batchsize: ' + str(self.batch_size), '\n- epochs: ' + str(self.epochs), '\n- initializer_seed: ' + str(self.initializer_seed))
            print('\n AUC Model: ', self.AUC)

    def inputs(self):
        print('\n[INFO] INPUTS')
        self.topt = str(input("Optimizer:  - (options: Adadelta, Adam, SGD, Adagrad)"))
        self.top_m = str(input("Top-model:  - (options: GMP, GAP, FD)"))
        self.lr = float(input("Learning-rate: "))
        self.batch_size = int(input("Batch-size: "))
        self.epochs = int(input("Epoch: "))
        
    def My_model_runall(self):
        
        if self.I_want_to_train_models == 'Yes': self.inputs()
            
        print('\n[INFO] LOADING DATA')
        self.load_data()
        
        if self.I_want_to_train_models == 'Yes':
            
            print('\n[INFO] CREATE MODEL')
            self.base_model()
            self.model_top()

            print('\n[INFO] COMPILE MODEL')
            self.model_compilation()

            print('\n[INFO] TRAINING MODEL')
            self.model_fit()
            
        
        print('\n[INFO] EVALUATING MODEL')
        self.model_predict()
        
        print('\n[INFO] REPORT')
        if self.I_want_to_train_models == 'Yes': self.print_data()

        
        # Clean GPU memory
        del self.model
        tf.keras.backend.clear_session()

In [ ]:
#INPUTS
itrainpredict = str(input("Do you want to train a model?:  - (options: No, Yes)"))
itype_image = str(input("3D Images or 2D Images:  - (options: 2D, 3D)"))
imodel_conv = str(input("What type of model?:  - (options: CSIC, VGG16)"))
imodel_type=  str(input("Hybrid or only Images?:  - (options: Hybrid, Images)"))
    
if (itype_image == '3D' and imodel_conv != 'CSIC'):
    print('Model unavailable')
    #sys.exit()    

    
My_model= My_Custom_Model(
                                                            I_want_to_train_models = itrainpredict,

                                                            path_csv= '/home/jovyan/IMAGE_PREPROCESSING_2D_3D/DIGITAL_CSIC_ICH/ICH_DETECTION/'+itype_image+'/CSV/' ,
                                                            path_images= '/home/jovyan/GITHUB_ICH_PROGNOSIS/DIGITAL CSIC  ICH/ICH_DETECTION/'+itype_image+'/PREPROCESSED IMAGES/ALL_IMAGES/',

                                                            img_width= 128,
                                                            img_height= 128,
                                                            dataAug = False,

                                                            #Hyperparameters
                                                            loss_function= 'binary_crossentropy',
                                                            
                                                            #Model
                                                            type_image = itype_image,
                                                            model_conv= imodel_conv, 
                                                            model_type= imodel_type, 
                    
                                                            freeze= None, #Just for vgg16 --> True or False or None
                                                            drop = 0.5,
                                                            neurons_last_layer= 1,
                                                            initializer_seed= 2,
                                                            metrics= ['accuracy'],

                                                            model_directory= '/home/jovyan/ICH_DETECTION/RESULTS/',
                                                            patience= 5,

                                                            print_report= False,                                                            
                        
                                                            p_to_save = 0.6,
                        )


My_model.My_model_runall()